In [14]:
from __future__ import print_function
from beakerx import *
import pandas as pd
import sys
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
import dalmatian as dm
import numpy as np

sys.path.insert(0, '../')
from src.CCLE_postp_function import *
sys.path.insert(0, '../../JKBio/')
import Datanalytics as da
import TerraFunction as terra

from taigapy import TaigaClient
from IPython.display import Image, display
tc = TaigaClient()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [7]:
refnamespace="broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline"

# Downloading from terra

In [8]:
refwm= dm.WorkspaceManager(refnamespace)

Let's get samples from when we made the switch to CGA only data

In [123]:
res = refwm.get_sample_sets().loc['DM19Q2_COMPLETE_SET']
res

aggregateMAF                                                                      nan
filtered_CGA_MAF_aggregated         gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b7...
passedCGA_filteredMAF_aggregated                                                  nan
samples                             [ibm_ACH-002446, ibm_ACH-000658, ibm_ACH-00180...
unfiltered_CGA_MAF_aggregated       gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b7...
Name: 19Q3, dtype: object

In [10]:
filtered = res['passedCGA_filteredMAF_aggregated']
unfiltered = res['unfiltered_CGA_MAF_aggregated']

! gsutil cp $filtered "temp/mutation_filtered_terra_merged.txt"
! gsutil cp $unfiltered "temp/mutation_unfiltered_terra_merged.txt"

Copying gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/aa8cbc26-e1f1-4100-b424-e2a461d6f10d/aggregateMAFs_selectFields_workflow/f25a70f4-56e3-4e3c-87f4-75ad36c090f4/call-aggregateMAFs_selectFields/DM19Q2_COMPLETE_SET.mergedMAF.txt...
- [1 files][ 92.3 MiB/ 92.3 MiB]    4.3 MiB/s                                   
Operation completed over 1 objects/92.3 MiB.                                     
Copying gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/7bf15e7f-69d6-474c-b335-2c9848cf018c/aggregateMAFs_selectFields_workflow/8961929c-7fb0-4077-82cd-7264015c46a7/call-aggregateMAFs_selectFields/attempt-2/DM19Q2_COMPLETE_SET.mergedMAF.txt...
Resuming download for temp/mutation_unfiltered_terra_merged.txt component 0     
Resuming download for temp/mutation_unfiltered_terra_merged.txt component 1
Resuming download for temp/mutation_unfiltered_terra_merged.txt component 3
Resuming download for temp/mutation_unfiltered_terra_merged.txt component 2
| [1 files][858.2 MiB/858.2 MiB]    3.5 

let's also look at the latest data version to see if the problem comes from here

In [10]:
release = pd.read_csv('../temp/mutations.19Q3.tsv', sep='\t')

/Users/jeremie/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3,19,25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
len(release)

1277316

In [19]:
filtered = pd.read_csv('../temp/mutation_filtered_terra_merged.txt', sep='\t')
unfiltered = pd.read_csv('../temp/mutation_unfiltered_terra_merged.txt', sep='\t')

/Users/jeremie/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,43,44,51,60) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
len(filtered)

411053

let's only use the same set of columns for comparison purposes

In [ ]:
unfiltered = unfiltered[set(unfiltered.columns.tolist()) & set(filtered.columns.tolist())]
filtered = filtered[unfiltered.columns]

## How much genome change non overlap do we have to the filtered version?

{{b}}

In [41]:
b = len(set(filtered.Genome_Change.tolist())) - len(set(unfiltered.Genome_Change.tolist()) & set(filtered.Genome_Change.tolist()))
b

0

## how much genome change do we have in duplicate? 
R : {{a*100}}%

In [40]:
a = (len(filtered) - len(set(filtered.Genome_Change.tolist()))) / len(filtered)
a

0.0755741960282494

does not seem to be a 19Q3 problem

In [47]:
len(filtered[filtered.Hugo_Symbol=='TP53']) - len(release[release.Hugo_Symbol=='TP53'])

-809

In [45]:
set(filtered[filtered.Hugo_Symbol=='TP53'].Protein_Change) - set(release[release.Hugo_Symbol=='TP53'].Protein_Change)

{'p.PSS126fs'}

In [49]:
filtered.columns

Index(['t_ref_count', 't_alt_count', 'Genome_Change', 'Chromosome',
       'Codon_Change', 'Protein_Change', 'cDNA_Change', 'dbSNP_RS',
       'Start_position', 'End_position', 'dbSNP_Val_Status', 'Strand',
       'Center', 'Hugo_Symbol', 'Variant_Classification', 'Reference_Allele',
       'Tumor_Sample_Barcode', 'Entrez_Gene_Id', 'Tumor_Seq_Allele2',
       'Variant_Type', 'tumor_f', 'Annotation_Transcript', 'Tumor_Seq_Allele1',
       'NCBI_Build'],
      dtype='object')

In [50]:
filtered.iloc[0].values

array([18.0, 20, 'g.chr1:7879351G>A', '1', 'c.(1528-1530)tGt>tAt',
       'p.C510Y', 'c.1529G>A', 'rs376085980', 7879351, 7879351, nan,
       '__UNKNOWN__', '__UNKNOWN__', 'PER3', 'Missense_Mutation', 'G',
       'ccle2_ACH-000011', 8863, 'A', 'SNP', 0.526316,
       'ENST00000377532.3', 'G', '__UNKNOWN__'], dtype=object)

In [27]:
release.columns

Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'NCBI_Build', 'Chromosome',
       'Start_position', 'End_position', 'Strand', 'Variant_Classification',
       'Variant_Type', 'Reference_Allele', 'Tumor_Seq_Allele1', 'dbSNP_RS',
       'dbSNP_Val_Status', 'Genome_Change', 'Annotation_Transcript',
       'Tumor_Sample_Barcode', 'cDNA_Change', 'Codon_Change', 'Protein_Change',
       'isDeleterious', 'isTCGAhotspot', 'TCGAhsCnt', 'isCOSMIChotspot',
       'COSMIChsCnt', 'ExAC_AF', 'CGA_WES_AC', 'SangerWES_AC',
       'SangerRecalibWES_AC', 'RNAseq_AC', 'HC_AC', 'RD_AC', 'WGS_AC'],
      dtype='object')

In [ ]:
filtered[filtered.Protein_Change=='p.PSS126fs'].values

In [115]:
set(unfiltered[unfiltered.Hugo_Symbol=='TP53'].Protein_Change)

{nan,
 'p.157_158insL',
 'p.173_174VR>G',
 'p.282_283RR>R',
 'p.A119fs',
 'p.A159D',
 'p.A159P',
 'p.A159V',
 'p.A161D',
 'p.A161T',
 'p.A347A',
 'p.A347V',
 'p.C124*',
 'p.C135F',
 'p.C135R',
 'p.C135Y',
 'p.C141W',
 'p.C141Y',
 'p.C176F',
 'p.C176S',
 'p.C176fs',
 'p.C238G',
 'p.C238R',
 'p.C238S',
 'p.C238Y',
 'p.C242C',
 'p.C242F',
 'p.C242S',
 'p.C242W',
 'p.C275G',
 'p.C277F',
 'p.D259Y',
 'p.D281N',
 'p.D49H',
 'p.E11Q',
 'p.E180fs',
 'p.E204*',
 'p.E204fs',
 'p.E224D',
 'p.E224E',
 'p.E258*',
 'p.E258K',
 'p.E271K',
 'p.E285K',
 'p.E285V',
 'p.E286K',
 'p.E287*',
 'p.E287D',
 'p.E28Q',
 'p.E298*',
 'p.E326K',
 'p.E326fs',
 'p.E339*',
 'p.E349fs',
 'p.E51*',
 'p.E51E',
 'p.E56*',
 'p.EG198fs',
 'p.ERFEM336fs',
 'p.F113S',
 'p.G105R',
 'p.G154V',
 'p.G187C',
 'p.G199*',
 'p.G199G',
 'p.G244A',
 'p.G244C',
 'p.G244D',
 'p.G245C',
 'p.G245D',
 'p.G245V',
 'p.G262V',
 'p.G266E',
 'p.G266R',
 'p.G279fs',
 'p.G334V',
 'p.H178D',
 'p.H179Q',
 'p.H179R',
 'p.H193P',
 'p.H193R',
 'p.H193

In [53]:
set(filtered[filtered.Hugo_Symbol=='TP53'].Protein_Change)

{nan,
 'p.157_158insL',
 'p.173_174VR>G',
 'p.282_283RR>R',
 'p.A119fs',
 'p.A159D',
 'p.A159P',
 'p.A159V',
 'p.A161D',
 'p.A161T',
 'p.A347A',
 'p.A347V',
 'p.C124*',
 'p.C135F',
 'p.C135R',
 'p.C135Y',
 'p.C141W',
 'p.C141Y',
 'p.C176F',
 'p.C176S',
 'p.C176fs',
 'p.C238G',
 'p.C238R',
 'p.C238S',
 'p.C238Y',
 'p.C242F',
 'p.C242S',
 'p.C242W',
 'p.C275G',
 'p.C277F',
 'p.D259Y',
 'p.D281N',
 'p.D49H',
 'p.E11Q',
 'p.E180fs',
 'p.E204*',
 'p.E204fs',
 'p.E224D',
 'p.E224E',
 'p.E258*',
 'p.E258K',
 'p.E271K',
 'p.E285K',
 'p.E285V',
 'p.E286K',
 'p.E287*',
 'p.E287D',
 'p.E28Q',
 'p.E298*',
 'p.E326K',
 'p.E326fs',
 'p.E339*',
 'p.E349fs',
 'p.E51*',
 'p.E56*',
 'p.EG198fs',
 'p.ERFEM336fs',
 'p.F113S',
 'p.G105R',
 'p.G154V',
 'p.G187C',
 'p.G199*',
 'p.G199G',
 'p.G244A',
 'p.G244C',
 'p.G244D',
 'p.G245C',
 'p.G245D',
 'p.G245V',
 'p.G262V',
 'p.G266E',
 'p.G266R',
 'p.G279fs',
 'p.G334V',
 'p.H178D',
 'p.H179Q',
 'p.H179R',
 'p.H193P',
 'p.H193R',
 'p.H193Y',
 'p.H296Y',
 'p.HER

In [51]:
set(release[release.Hugo_Symbol=='TP53'].Protein_Change)

{nan,
 'p.*W262fs',
 'p.146_149WVDS>C',
 'p.157_158insL',
 'p.165_166insYKQ',
 'p.173_174VR>G',
 'p.239_239N>NN',
 'p.247_248NR>KW',
 'p.249_250insR',
 'p.282_283RR>R',
 'p.52_52Q>P*',
 'p.A119fs',
 'p.A138P',
 'p.A138fs',
 'p.A159D',
 'p.A159P',
 'p.A159V',
 'p.A161D',
 'p.A161T',
 'p.A189A',
 'p.A307V',
 'p.A347A',
 'p.A347P',
 'p.A347V',
 'p.A70A',
 'p.A74S',
 'p.A76fs',
 'p.APA74fs',
 'p.APSWPLSSS88fs',
 'p.C124*',
 'p.C124R',
 'p.C135F',
 'p.C135R',
 'p.C135Y',
 'p.C135fs',
 'p.C141W',
 'p.C141Y',
 'p.C176F',
 'p.C176S',
 'p.C176W',
 'p.C176Y',
 'p.C176fs',
 'p.C229fs',
 'p.C238F',
 'p.C238G',
 'p.C238R',
 'p.C238S',
 'p.C238Y',
 'p.C242F',
 'p.C242R',
 'p.C242S',
 'p.C242W',
 'p.C242Y',
 'p.C275F',
 'p.C275G',
 'p.C275Y',
 'p.C277F',
 'p.CNSSC238fs',
 'p.D126Y',
 'p.D259N',
 'p.D259V',
 'p.D259Y',
 'p.D281G',
 'p.D281N',
 'p.D281Y',
 'p.D48N',
 'p.D49H',
 'p.DD48fs',
 'p.E11Q',
 'p.E171*',
 'p.E171del',
 'p.E180*',
 'p.E180K',
 'p.E180fs',
 'p.E204*',
 'p.E204fs',
 'p.E224*',
 'p

## Looking at data from 19Q1 to see exactly the missign mutations

In [15]:
new = tc.get(name='depmap-mutation-calls-9be3', version=15, file='depmap_19Q3_mutation_calls')
prev = tc.get(name='depmap-mutation-calls-9be3', version=11, file='depmap_19Q1_mutation_calls')

In [28]:
prev= prev.drop('Unnamed: 0',1)

In [16]:
prev.columns

Index(['Unnamed: 0', 'Hugo_Symbol', 'Entrez_Gene_Id', 'NCBI_Build',
       'Chromosome', 'Start_position', 'End_position', 'Strand',
       'Variant_Classification', 'Variant_Type', 'Reference_Allele',
       'Tumor_Seq_Allele1', 'dbSNP_RS', 'dbSNP_Val_Status', 'Genome_Change',
       'Annotation_Transcript', 'Tumor_Sample_Barcode', 'cDNA_Change',
       'Codon_Change', 'Protein_Change', 'isDeleterious', 'isTCGAhotspot',
       'TCGAhsCnt', 'isCOSMIChotspot', 'COSMIChsCnt', 'ExAC_AF', 'VA_WES_AC',
       'CGA_WES_AC', 'SangerWES_AC', 'SangerRecalibWES_AC', 'RNAseq_AC',
       'HC_AC', 'RD_AC', 'WGS_AC', 'Variant_annotation', 'DepMap_ID'],
      dtype='object')

lets only look at the data of cell lines that we had in VA+ CGA vs CGA

In [17]:
newp = new[new.DepMap_ID.isin(set(prev.DepMap_ID))]

In [43]:
unfiltered['DepMap_ID'] =[i.split('_')[1] for i in unfiltered['Tumor_Sample_Barcode']]

In [44]:
unfiltered = unfiltered[unfiltered.DepMap_ID.isin(set(prev.DepMap_ID))]

we still have more data on the unfiltered version (seem to make sense)

In [20]:
len(unfiltered) - len(prev)

65763

there seem to be a lot less data in the new version compared to the old one 

In [21]:
len(newp) - len(prev)

-38107

In [56]:
len(newp[newp.Hugo_Symbol=='TP53'])

1274

In [22]:
len(prev[prev.Hugo_Symbol=='TP53'])

1280

there is cell lines for which there is mutations that have been called in VA +CGA but not in CGA. And here are the genome changes that have never been called in the CGA only version

In [23]:
set(prev[prev.Hugo_Symbol=='TP53'].DepMap_ID) - set(newp[newp.Hugo_Symbol=='TP53'].DepMap_ID) 

{'ACH-000036', 'ACH-001639'}

In [24]:
other = set(prev[prev.Hugo_Symbol=='TP53'].Genome_Change) - set(newp[newp.Hugo_Symbol=='TP53'].Genome_Change) 
other

{'g.chr17:7573986_7573987GG>AA',
 'g.chr17:7577066_7577067TT>AA',
 'g.chr17:7577094G>A',
 'g.chr17:7577117_7577118AC>CA',
 'g.chr17:7577538C>T',
 'g.chr17:7577548C>T',
 'g.chr17:7577558_7577559GG>AA',
 'g.chr17:7579311_7579312CC>TT'}

In [57]:
cgamva = set(newp[newp.Hugo_Symbol=='TP53'].Genome_Change) - set(prev[prev.Hugo_Symbol=='TP53'].Genome_Change) 
cgamva

{nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan}

In [62]:
new[new.Hugo_Symbol=='TP53'][new.Genome_Change.isin(cgamva)]

/Users/jeremie/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,ExAC_AF,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Variant_annotation,DepMap_ID
96,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,0.000058,NaN,20:0,20:0,302:4,70:1,NaN,19:0,other non-conserving,ACH-000001
3774,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,0.000058,0:0,NaN,NaN,594:4,72:0,NaN,NaN,other non-conserving,ACH-000013
7261,TP53,7157,37,17,7577094,7577094,+,Missense_Mutation,SNP,G,...,0.000016,0:0,NaN,NaN,412:3,46:1,339:2,NaN,other non-conserving,ACH-000022
7533,TP53,7157,37,17,7577094,7577094,+,Missense_Mutation,SNP,G,...,0.000016,NaN,34:0,34:0,750:1,30:0,325:3,20:0,other non-conserving,ACH-000023
9837,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,0.000058,NaN,67:0,67:0,836:3,52:0,NaN,44:0,other non-conserving,ACH-000030
15493,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,0.000058,0:0,28:0,28:0,222:4,96:0,NaN,NaN,other non-conserving,ACH-000042
18271,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,0.000058,0:0,NaN,NaN,NaN,NaN,NaN,NaN,other non-conserving,ACH-000055
27382,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,0.000058,NaN,22:0,19:0,270:12,61:0,NaN,NaN,other non-conserving,ACH-000067
31873,TP53,7157,37,17,7577548,7577548,+,Missense_Mutation,SNP,C,...,0.000008,0:0,NaN,NaN,612:4,57:0,NaN,NaN,other non-conserving,ACH-000089
37100,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,0.000058,0:0,NaN,NaN,521:2,55:0,NaN,NaN,other non-conserving,ACH-000110


let's look at the tp53 mutations with this particular genome change.

In [79]:
a = pd.read_csv('../../../../../new.tsv', sep='\t')
b = a[a.Hugo_Symbol=='TP53']

In [85]:
b.T.to_csv('new.csv')

In [72]:
prev.columns.tolist()

['Hugo_Symbol',
 'Entrez_Gene_Id',
 'NCBI_Build',
 'Chromosome',
 'Start_position',
 'End_position',
 'Strand',
 'Variant_Classification',
 'Variant_Type',
 'Reference_Allele',
 'Tumor_Seq_Allele1',
 'dbSNP_RS',
 'dbSNP_Val_Status',
 'Genome_Change',
 'Annotation_Transcript',
 'Tumor_Sample_Barcode',
 'cDNA_Change',
 'Codon_Change',
 'Protein_Change',
 'isDeleterious',
 'isTCGAhotspot',
 'TCGAhsCnt',
 'isCOSMIChotspot',
 'COSMIChsCnt',
 'ExAC_AF',
 'VA_WES_AC',
 'CGA_WES_AC',
 'SangerWES_AC',
 'SangerRecalibWES_AC',
 'RNAseq_AC',
 'HC_AC',
 'RD_AC',
 'WGS_AC',
 'Variant_annotation',
 'DepMap_ID']

In [76]:
prev[prev.Hugo_Symbol=='TP53'][prev.Genome_Change.isin(other)]

/Users/jeremie/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,VA_WES_AC,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Variant_annotation,DepMap_ID
96,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,NaN,NaN,20:0,20:0,302:4,70:1,NaN,19:0,other non-conserving,ACH-000001
3907,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,36:1,0:0,NaN,NaN,594:4,72:0,NaN,NaN,other non-conserving,ACH-000013
7507,TP53,7157,37,17,7577094,7577094,+,Missense_Mutation,SNP,G,...,420:0,0:0,NaN,NaN,412:3,46:1,339:2,NaN,other non-conserving,ACH-000022
7802,TP53,7157,37,17,7577094,7577094,+,Missense_Mutation,SNP,G,...,NaN,NaN,34:0,34:0,750:1,30:0,325:3,20:0,other non-conserving,ACH-000023
10252,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,NaN,NaN,67:0,67:0,836:3,52:0,NaN,44:0,other non-conserving,ACH-000030
16019,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,48:1,0:0,28:0,28:0,222:4,96:0,NaN,NaN,other non-conserving,ACH-000042
18889,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,6:72,0:0,NaN,NaN,NaN,NaN,NaN,NaN,other non-conserving,ACH-000055
28063,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,NaN,NaN,22:0,19:0,270:12,61:0,NaN,NaN,other non-conserving,ACH-000067
32748,TP53,7157,37,17,7577548,7577548,+,Missense_Mutation,SNP,C,...,46:1,0:0,NaN,NaN,612:4,57:0,NaN,NaN,other non-conserving,ACH-000089
38222,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,19:0,0:0,NaN,NaN,521:2,55:0,NaN,NaN,other non-conserving,ACH-000110


In [32]:
tp53prev = prev[prev.Hugo_Symbol=='TP53']

[autoreload of ipywidgets.widgets failed: Traceback (most recent call last):
  File "/Users/jeremie/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 244, in check
    superreload(m, reload, self.old_objects)
  File "/Users/jeremie/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 378, in superreload
    module = reload(module)
  File "/Users/jeremie/anaconda3/envs/py36/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/Users/jeremie/anaconda3/envs/py36/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/jeremie/anaconda3/envs/py36/lib/python3.6/site-packages/ipywidgets/widgets/__init__.py", line 21, in <module>

In [34]:
newp[newp.Hugo_Symbol=='TP53'][newp.Genome_Change.isin(set(tp53prev.Genome_Change)) & newp.DepMap_ID.isin(set(tp53prev.DepMap_ID))]

/Users/jeremie/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,ExAC_AF,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Variant_annotation,DepMap_ID
581,TP53,7157,37,17,7578239,7578239,+,Nonsense_Mutation,SNP,C,...,NaN,NaN,NaN,NaN,8:1,NaN,NaN,NaN,damaging,ACH-000003
800,TP53,7157,37,17,7578532,7578532,+,Missense_Mutation,SNP,A,...,NaN,NaN,NaN,23:0,362:0,25:0,NaN,NaN,other non-conserving,ACH-000004
1091,TP53,7157,37,17,7578532,7578532,+,Missense_Mutation,SNP,A,...,NaN,153:0,NaN,NaN,340:0,22:0,274:0,23:0,other non-conserving,ACH-000005
1341,TP53,7157,37,17,7577120,7577120,+,Missense_Mutation,SNP,C,...,0.000025,NaN,22:0,23:0,381:4,11:0,235:1,NaN,other non-conserving,ACH-000006
2577,TP53,7157,37,17,7578239,7578239,+,Nonsense_Mutation,SNP,C,...,NaN,NaN,28:0,28:0,10:2,240:0,NaN,27:0,damaging,ACH-000009
2898,TP53,7157,37,17,7578500,7578500,+,Nonsense_Mutation,SNP,G,...,0.000008,NaN,NaN,NaN,22:0,NaN,NaN,NaN,damaging,ACH-000010
3524,TP53,7157,37,17,7578195,7578197,+,In_Frame_Del,DEL,CAC,...,NaN,NaN,33:2,40:2,192:0,83:8,NaN,44:8,other non-conserving,ACH-000012
4217,TP53,7157,37,17,7577098,7577098,+,Missense_Mutation,SNP,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:32,other non-conserving,ACH-000014
4792,TP53,7157,37,17,7578500,7578500,+,Nonsense_Mutation,SNP,G,...,0.000008,NaN,14:0,15:0,17:3,82:2,428:251,NaN,damaging,ACH-000015
5344,TP53,7157,37,17,7578406,7578406,+,Missense_Mutation,SNP,C,...,0.000008,248:0,NaN,NaN,248:5,33:0,NaN,NaN,other non-conserving,ACH-000017


In [36]:
found = []
for i, v in newp[newp.Hugo_Symbol=='TP53'].iterrows():
    print(i,end='\r')
    for k, val in prev[prev.Hugo_Symbol=='TP53'].iterrows():
        if val.Genome_Change == v.Genome_Change:
            if val.DepMap_ID == v.DepMap_ID:
                found.append(i)
                break

In [40]:
innewnotinprev = newp[~newp.index.isin(found)][newp.Hugo_Symbol=='TP53']

/Users/jeremie/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [41]:
len(innewnotinprev)

97

In [114]:
innewnotinprev.isCOSMIChotspot.sum()

0

In [157]:
len([i for i in inprevnotinnew.VA_WES_AC.values if type(i) is str])

56

In [46]:
found = []
for k, val in prev[prev.Hugo_Symbol=='TP53'].iterrows():
    print(k,end='\r')
    for i, v in unfiltered[unfiltered.Hugo_Symbol=='TP53'].iterrows():
        if val.Genome_Change == v.Genome_Change:
            if val.DepMap_ID == v.DepMap_ID:
                found.append(k)
                break

In [52]:
inprevnotinunfiltered = prev[~prev.index.isin(found)][prev.Hugo_Symbol=='TP53']

/Users/jeremie/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [53]:
len(inprevnotinunfiltered)

845

In [54]:
inprevnotinunfiltered

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,VA_WES_AC,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Variant_annotation,DepMap_ID
96,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,NaN,NaN,20:0,20:0,302:4,70:1,NaN,19:0,other non-conserving,ACH-000001
582,TP53,7157,37,17,7578239,7578239,+,Nonsense_Mutation,SNP,C,...,NaN,NaN,NaN,NaN,8:1,NaN,NaN,NaN,damaging,ACH-000003
801,TP53,7157,37,17,7578532,7578532,+,Missense_Mutation,SNP,A,...,NaN,NaN,NaN,23:0,362:0,25:0,NaN,NaN,other non-conserving,ACH-000004
1390,TP53,7157,37,17,7577120,7577120,+,Missense_Mutation,SNP,C,...,NaN,NaN,22:0,23:0,381:4,11:0,235:1,NaN,other non-conserving,ACH-000006
2676,TP53,7157,37,17,7578239,7578239,+,Nonsense_Mutation,SNP,C,...,NaN,NaN,28:0,28:0,10:2,240:0,NaN,27:0,damaging,ACH-000009
2997,TP53,7157,37,17,7578500,7578500,+,Nonsense_Mutation,SNP,G,...,NaN,NaN,NaN,NaN,22:0,NaN,NaN,NaN,damaging,ACH-000010
3646,TP53,7157,37,17,7578195,7578197,+,In_Frame_Del,DEL,CAC,...,NaN,NaN,33:2,40:2,192:0,83:8,NaN,44:8,other non-conserving,ACH-000012
3907,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,36:1,0:0,NaN,NaN,594:4,72:0,NaN,NaN,other non-conserving,ACH-000013
4360,TP53,7157,37,17,7577098,7577098,+,Missense_Mutation,SNP,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:32,other non-conserving,ACH-000014
4935,TP53,7157,37,17,7578500,7578500,+,Nonsense_Mutation,SNP,G,...,NaN,NaN,14:0,15:0,17:3,82:2,428:251,NaN,damaging,ACH-000015


In [55]:
found = []
for k, val in prev[prev.Hugo_Symbol=='TP53'].iterrows():
    print(k,end='\r')
    for i, v in newp[newp.Hugo_Symbol=='TP53'].iterrows():
        if val.Genome_Change == v.Genome_Change:
            if val.DepMap_ID == v.DepMap_ID:
                found.append(k)
                break

In [49]:
inprevnotinnew = prev[~prev.index.isin(found)][prev.Hugo_Symbol=='TP53']

/Users/jeremie/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [50]:
len(inprevnotinnew)

845

In [113]:
inprevnotinnew.isCOSMIChotspot

96         True
3907       True
7507       True
7802       True
10252      True
11600      True
16019      True
18889      True
28063      True
32748      True
38222      True
39289      True
40262      True
46114      True
46389      True
47082      True
61447      True
61448      True
73821      True
76708      True
79462      True
83254      True
90778      True
93108      True
93437      True
104133     True
108207     True
110180     True
122527     True
122922     True
           ... 
1043889    True
1062330    True
1067837    True
1076521    True
1080008    True
1092656    True
1101249    True
1102406    True
1107763    True
1110117    True
1113741    True
1115475    True
1115837    True
1121896    True
1123132    True
1146095    True
1146713    True
1154436    True
1185132    True
1186555    True
1197473    True
1197900    True
1198144    True
1199036    True
1219800    True
1219801    True
1224520    True
1234018    True
1244321    True
1260959    True
Name: isCOSMIChotspot, L

In [51]:
inprevnotinnew

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,VA_WES_AC,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Variant_annotation,DepMap_ID
96,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,NaN,NaN,20:0,20:0,302:4,70:1,NaN,19:0,other non-conserving,ACH-000001
582,TP53,7157,37,17,7578239,7578239,+,Nonsense_Mutation,SNP,C,...,NaN,NaN,NaN,NaN,8:1,NaN,NaN,NaN,damaging,ACH-000003
801,TP53,7157,37,17,7578532,7578532,+,Missense_Mutation,SNP,A,...,NaN,NaN,NaN,23:0,362:0,25:0,NaN,NaN,other non-conserving,ACH-000004
1390,TP53,7157,37,17,7577120,7577120,+,Missense_Mutation,SNP,C,...,NaN,NaN,22:0,23:0,381:4,11:0,235:1,NaN,other non-conserving,ACH-000006
2676,TP53,7157,37,17,7578239,7578239,+,Nonsense_Mutation,SNP,C,...,NaN,NaN,28:0,28:0,10:2,240:0,NaN,27:0,damaging,ACH-000009
2997,TP53,7157,37,17,7578500,7578500,+,Nonsense_Mutation,SNP,G,...,NaN,NaN,NaN,NaN,22:0,NaN,NaN,NaN,damaging,ACH-000010
3646,TP53,7157,37,17,7578195,7578197,+,In_Frame_Del,DEL,CAC,...,NaN,NaN,33:2,40:2,192:0,83:8,NaN,44:8,other non-conserving,ACH-000012
3907,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,36:1,0:0,NaN,NaN,594:4,72:0,NaN,NaN,other non-conserving,ACH-000013
4360,TP53,7157,37,17,7577098,7577098,+,Missense_Mutation,SNP,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:32,other non-conserving,ACH-000014
4935,TP53,7157,37,17,7578500,7578500,+,Nonsense_Mutation,SNP,G,...,NaN,NaN,14:0,15:0,17:3,82:2,428:251,NaN,damaging,ACH-000015


In [85]:
np.where(((prev[prev.Hugo_Symbol=='TP53'].Genome_Change == \
newp[newp.Hugo_Symbol=='TP53'].Genome_Change) & \
         (newp[newp.Hugo_Symbol=='TP53'].DepMap_ID == \
         newp[newp.Hugo_Symbol=='TP53'].DepMap_ID)))

ValueError: Can only compare identically-labeled Series objects